In [12]:
pip install -r ../../requirements.txt

python(43647) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [13]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [14]:
from zkstats.core import create_dummy, verifier_define_calculation, prover_gen_settings, setup, prover_gen_proof, verifier_verify, get_data_commitment_maps

In [15]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')

=======================  ZK-STATS FLOW =======================

In [16]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

create_dummy(data_path, dummy_data_path)

In [17]:
scales = [6]
selected_columns = ['col_name']
commitment_maps = get_data_commitment_maps(data_path, scales)

In [18]:
# Verifier/ data consumer side:
# since have division, scale must be very LARGE, cant use just 0
# So, we allow error rate to be 10% instead of 1%
from zkstats.computation import computation_to_model, State


def computation(s: State, data: list[torch.Tensor]) -> torch.Tensor:
    x = data[0]
    return s.harmonic_mean(x)

error = 0.1
_, verifier_model = computation_to_model(computation, error)

verifier_define_calculation(dummy_data_path, selected_columns,sel_dummy_data_path,verifier_model, verifier_model_path)

In [19]:
# prover calculates settings, send to verifier
_, prover_model = computation_to_model(computation, error)

prover_gen_settings(data_path, selected_columns, sel_data_path, prover_model,prover_model_path, scales, "resources", settings_path)

==== Generate & Calibrate Setting ====
scale:  [6]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":6,"param_scale":6,"scale_rebase_multiplier":10,"lookup_range":[-184392,11264],"logrows":18,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":911,"total_const_size":4,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,6],"model_input_scales":[6],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":["Abs",{"Recip":{"scale":4096.0}},{"GreaterThan":{"a":0.0}}],"check_mode":"UNSAFE","version":"7.0.0","num_blinding_factors":null}


In [20]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

spawning module 0
spawning module 2


==== setting up ezkl ====


spawning module 0
spawning module 2


Time setup: 25.911302089691162 seconds
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 45.21875
==== Generating Proof ====


spawning module 0
spawning module 2


proof:  {'instances': [[[11712583131456230033, 7998639690917952521, 17628064685104548320, 3364894255067083796], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [10409296726488761395, 15557203226960814343, 16692342436085548322, 2420882828764455196]]], 'proof': '2deccea5835cd138662b01fc5d10a983847f69015420993499d62bff137dec7b1e177255ff696c06524172ee1716ede8089af624975f894e21580ad4560b8e8225a316b072f7a939d24a007b31c15b6c2fbac7da2d2638083186f8a5716e7e4024f1ae907c216cad86ba0efdb467ee5883464227d979b87894b7f50b64b9dc7319fc4b8947ee68fe3c00d64012380f6c8f5f8cdccddd1532ddba660ee6633d0e19bac10b8485d21f612aaec0ec3c79133a15c45e4d46708a06dc507af065b6891a51e15d7095073a6131ca2a7389e9804eb2c82ebe57dc5bb1dcc720e5d8a4d028d29bd40913b577bfb9a523e3f8b9c6e08ddb7fd2d9e9fdcae0e0d5f742a49b0194530171682c571738837043aa6802def5e52e7a2d7a8606db2993badc7eb5031eff3ad8a2a4f19724621e6f7cca94bc454b6b84e0d36c651a61c4b8e0086719725982afb11367f81e4f13126cbbd25a845865cf3d820aa04246b16c24

In [21]:
# Verifier verifies
res = verifier_verify(proof_path, settings_path, vk_path, selected_columns, commitment_maps)
print("Verifier gets result:", res)

Verifier gets result: [45.21875]
